# Credit Risk Scorecard Demo

## Before you begin
To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also create a documentation project on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
# %pip install --upgrade validmind

## Initialize the client library
In a browser, go to the Client Integration page of your documentation project and click Copy to clipboard next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

This step requires a documentation project. Learn how you can create one.

Next, replace this placeholder with your own code snippet:

In [2]:
import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "2494c3838f48efe590d531bfe225d90b",
  api_secret = "4f692f8161f128414fef542cab2a4e74834c75d01b3a8e088a1834f2afcfe838",
  project = "clk00h0u800x9qjy67gduf5om"
)

2023-08-03 15:39:10,262 - INFO(validmind.api_client): Connected to ValidMind. Project: [6] Credit Risk Scorecard - Initial Validation (clk00h0u800x9qjy67gduf5om)


## Use case

### Setup

#### Introduction

The **Credit risk Scorecard** model created from the Lending Club dataset is instrumental in computing the Probability of Default (PD), a key factor in ECL calculations. This scorecard assesses several credit characteristics of potential borrowers, like their credit history, income, outstanding debts, and more, each of which is assigned a specific score. By combining these scores, we derive a total score for each borrower, which translates into an estimated Point-in-Time (PiT) PD. The PiT PD reflects the borrower's likelihood of default at a specific point in time, accounting for both current and foreseeable future conditions.

Additionally, for a holistic view of credit risk, it's essential to estimate the Lifetime PD. The Lifetime PD, as the name suggests, predicts the borrower's likelihood of default throughout the life of the exposure, taking into account potential future changes in the economic and financial conditions.

#### Import Libraries

In [3]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

# Standard library imports
import re
import pickle
from datetime import datetime
from typing import List

# Data handling and analysis imports
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Visualization imports
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# File handling import
import zipfile


#### Processing Functions

In [4]:
def save_model_and_df(model, df, base_filename):
    """Save a model and a dataframe with a timestamp in the filename"""
    # Get current date and time
    now = datetime.now()

    # Convert the current date and time to string
    timestamp_str = now.strftime("%Y%m%d_%H%M%S")

    filename = f'{base_filename}_{timestamp_str}.pkl'

    # Save the model and dataframe
    with open(filename, 'wb') as file:
        pickle.dump((model, df), file)
        
    print(f"Model and dataframe saved as {filename}")


In [5]:
def get_numerical_columns(df):
        numerical_columns = df.select_dtypes(
            include=["int", "float", "uint"]
        ).columns.tolist()
        return numerical_columns

def get_categorical_columns(df):
        categorical_columns = df.select_dtypes(
            include=["object", "category"]
        ).columns.tolist()
        return categorical_columns

In [6]:
def add_target_column(df, target_column):
    # Assuming the column name is 'loan_status'
    df[target_column] = df['loan_status'].apply(lambda x: 0 if x == "Fully Paid" else 1 if x == "Charged Off" else np.nan)
    # Remove rows where the target column is NaN
    df = df.dropna(subset=[target_column])
    # Convert target column to integer
    df[target_column] = df[target_column].astype(int)
    return df

In [7]:
unused_variables = ["id", "member_id", "funded_amnt", "emp_title", "url", "desc", "application_type",
                    "title", "zip_code", "delinq_2yrs", "mths_since_last_delinq", "mths_since_last_record",
                    "revol_bal", "total_rec_prncp", "total_rec_late_fee", "recoveries", "out_prncp_inv", "out_prncp", 
                    "collection_recovery_fee", "next_pymnt_d", "initial_list_status", "pub_rec",
                    "collections_12_mths_ex_med", "policy_code", "acc_now_delinq", "pymnt_plan",
                    "tot_coll_amt", "tot_cur_bal", "total_rev_hi_lim", "last_pymnt_d", "last_credit_pull_d",
                    'earliest_cr_line', 'issue_d']

In [8]:
def variables_with_min_missing(df, min_missing_percentage):
    # Calculate the percentage of missing values in each column
    missing_percentages = df.isnull().mean() * 100

    # Get the variables where the percentage of missing values is greater than the specified minimum
    variables_to_drop = missing_percentages[missing_percentages > min_missing_percentage].index.tolist()

    # Also add any columns where all values are missing
    variables_to_drop.extend(df.columns[df.isnull().all()].tolist())

    # Remove duplicates (if any)
    variables_to_drop = list(set(variables_to_drop))

    return variables_to_drop

In [9]:
def clean_term_column(df, column):
    """
    Function to remove 'months' string from the 'term' column and convert it to categorical
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")
    
    df[column] = df[column].str.replace(' months', '')
    
    # Convert to categorical
    df[column] = df[column].astype('object')

def clean_emp_length_column(df, column):
    """
    Function to clean 'emp_length' column and convert it to categorical.
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")
    
    df[column] = df[column].replace('n/a', np.nan)
    df[column] = df[column].str.replace('< 1 year', str(0))
    df[column] = df[column].apply(lambda x: re.sub('\D', '', str(x)))
    df[column].fillna(value = 0, inplace=True)

    # Convert to categorical
    df[column] = df[column].astype('object')

def clean_inq_last_6mths(df, column):
    """
    Function to convert 'inq_last_6mths' column into categorical.
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")

    # Convert to categorical
    df[column] = df[column].astype('category')

In [10]:
def compute_outliers(series, threshold=1.5):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return series[(series < lower_bound) | (series > upper_bound)]

def remove_iqr_outliers(df, target_column, threshold=1.5):
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    num_cols.remove(target_column)  # Exclude target_column from numerical columns
    for col in num_cols:
        outliers = compute_outliers(df[col], threshold)
        df = df[~df[col].isin(outliers)]
    return df

In [11]:
import pandas as pd
import numpy as np

def class_binning(df, bin_mappings):
    df_new = df.copy()
    
    for feature, bins in bin_mappings.items():
        # Convert to numeric, turn non-numeric data into NaN
        df_new[feature] = pd.to_numeric(df_new[feature], errors='coerce')
        
        # Fill NaN with a default value
        df_new[feature].fillna(-1, inplace=True)
        
        # Bin the feature
        bins = [-np.inf] + bins + [np.inf]
        df_new[f'{feature}_bucket'] = pd.cut(df_new[feature], bins=bins, right=False, include_lowest=True)
        df_new.drop(columns=feature, inplace=True)
    
    return df_new


def class_coarsing(df, coarse_mappings):
    df_new = df.copy()

    for feature, coarse_bins in coarse_mappings.items():
        df_new[f'{feature}_coarse'] = df_new[feature]  # start with original categories

        for i, coarse_bin in enumerate(coarse_bins):
            df_new.loc[df_new[feature].isin(coarse_bin), f'{feature}_coarse'] = f'{feature}_group_{i}'

        df_new.drop(columns=feature, inplace=True)
    
    return df_new

In [12]:
def find_categorical_features(df):
    # Get the column names of features with the data type "category"
    categorical_features = df.select_dtypes(include='category').columns.tolist()

    return categorical_features


def convert_categorical_to_object(df):
    # Find the categorical features
    categorical_features = find_categorical_features(df)

    # Convert the categorical features to object type
    df[categorical_features] = df[categorical_features].astype(str)

In [13]:
def check_categories(woe_df, original_df):
    for feature in woe_df['Feature'].unique():
        woe_categories = woe_df[woe_df['Feature'] == feature]['Category'].unique()
        original_categories = original_df[feature].unique()
        
        # Check categories in WoE table that are not in original DataFrame
        for category in woe_categories:
            if category not in original_categories:
                print(f"Category '{category}' not found in feature '{feature}' in original DataFrame.")
                
        # Check categories in original DataFrame that are not in WoE table
        for category in original_categories:
            if category not in woe_categories:
                print(f"Category '{category}' in feature '{feature}' not found in WoE table.")

In [14]:
def woe_encoder(woe_df, original_df, target):
    # Create a new DataFrame with the same columns as original_df
    woe_encoded_df = pd.DataFrame(columns=original_df.columns, index=original_df.index)

    # Loop through each feature-category and assign the corresponding WoE value as float
    for feature in woe_df['Feature'].unique():
        # Check that the feature exists in the original DataFrame
        if feature not in original_df.columns:
            print(f"Feature {feature} not found in original DataFrame. Skipping...")
            continue

        feature_woe = woe_df[woe_df['Feature'] == feature]
        woe_dict = dict(zip(feature_woe['Category'], feature_woe['WoE']))

        # Check that the categories exist in the original DataFrame
        # Converting both to strings to avoid datatype issues
        original_categories = original_df[feature].astype(str).unique()
        woe_categories = feature_woe['Category'].astype(str).unique()
        
        # Two-way check:
        # 1. For each category in the original DataFrame, check if it exists in the WoE DataFrame
        missing_from_woe = [category for category in original_categories if category not in woe_categories]
        if missing_from_woe:
            print(f"Categories {missing_from_woe} from original DataFrame not found in WoE DataFrame for feature {feature}.")
            
        # 2. For each category in the WoE DataFrame, check if it exists in the original DataFrame
        missing_from_original = [category for category in woe_categories if category not in original_categories]
        if missing_from_original:
            print(f"Categories {missing_from_original} from WoE DataFrame not found in original DataFrame for feature {feature}.")
        
        # Also converting original dataframe feature to string before replacement
        woe_encoded_df[feature] = original_df[feature].astype(str).replace(woe_dict).astype(float)

    # Check that the target exists in the original DataFrame
    if target not in original_df.columns:
        print(f"Target {target} not found in original DataFrame. Returning None...")
        return None

    # Add the target column to the new DataFrame
    woe_encoded_df[target] = original_df[target]

    return woe_encoded_df

## Data Description

#### Import Raw Data

In [15]:
# Download the Lending Club dataset from a public S3 bucket
lending_club_url = "https://vmai.s3.us-west-1.amazonaws.com/datasets/lending_club_loan_data_2007_2014.csv"
df_raw = pd.read_csv(lending_club_url)

/var/folders/tn/rbr74q892k13m82y37y396h40000gn/T/ipykernel_92721/1775071480.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(lending_club_url)


#### Validate Raw Data

In [16]:
from validmind.vm_models.test_context import TestContext
from validmind.tests.data_validation.DescriptiveStatistics import DescriptiveStatistics

vm_df_raw = vm.init_dataset(dataset=df_raw)
test_context_raw = TestContext(dataset=vm_df_raw)

metric = DescriptiveStatistics(test_context_raw)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-03 15:41:20,587 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-03 15:41:20,587 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [17]:
from validmind.tests.data_validation.MissingValuesBarPlot import MissingValuesBarPlot

params = {"threshold": 70,
          "fig_height": 1100}

metric = MissingValuesBarPlot(test_context_raw, params)
metric.run()
await metric.result.log()
metric.result.show()

## Data Preparation

#### Process Raw Data

In [18]:
# Definition of default
target_column = 'default'
df_prep_1 = add_target_column(df_raw, target_column)

# Drop 'loan_status' variable 
df_prep_1.drop(columns='loan_status', axis=1, inplace=True)

# Remove unused variables
df_prep_1 = df_prep_1.drop(columns=unused_variables)

# Remove missing values
min_missing_count = 80
variables_to_drop = variables_with_min_missing(df_prep_1, min_missing_count)
df_prep_1.drop(columns=variables_to_drop, axis=1, inplace=True)
df_prep_1.dropna(axis=0, subset=["emp_length"], inplace=True)
df_prep_1.dropna(axis=0, subset=["revol_util"], inplace=True)

# Format variable types
clean_emp_length_column(df_prep_1, 'emp_length')
clean_term_column(df_prep_1, 'term')
clean_inq_last_6mths(df_prep_1, 'inq_last_6mths')

# Remove outliers
df_prep_1 = remove_iqr_outliers(df_prep_1, target_column, threshold=1.5)

/var/folders/tn/rbr74q892k13m82y37y396h40000gn/T/ipykernel_92721/668951511.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/tn/rbr74q892k13m82y37y396h40000gn/T/ipykernel_92721/2621629285.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Validate Prepared Data

In [19]:
from validmind.tests.data_validation.ClassImbalance import ClassImbalance

vm_df_prep_1 = vm.init_dataset(dataset=df_prep_1,
                        target_column=target_column)
test_context_prep_1 = TestContext(dataset=vm_df_prep_1)

metric = ClassImbalance(test_context_prep_1)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-03 15:41:32,133 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-03 15:41:32,134 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [20]:
from validmind.tests.data_validation.IQROutliersTable import IQROutliersTable

num_features = get_numerical_columns(df_prep_1)
params = {"num_features": num_features,
          "threshold": 1.5
        }

metric = IQROutliersTable(test_context_prep_1, params)
metric.run()
await metric.result.log()
metric.result.show()

In [21]:
from validmind.tests.data_validation.IQROutliersBarPlot import IQROutliersBarPlot

num_features = get_numerical_columns(df_prep_1)
params = {"num_features": num_features,
          "threshold": 1.5,
          "fig_width": 500}

metric = IQROutliersBarPlot(test_context_prep_1, params)
metric.run()
await metric.result.log()
metric.result.show()

## Data Sampling

#### Sampling Method

We employ stratified sampling to create our training and testing sets. Stratified sampling is particularly important in this context. When the `stratify = y` parameter is set, it ensures that the distribution of the target variable (`y`) in the test set is the same as that in the original dataset. 

This is crucial for maintaining a consistent representation of the target variable classes, especially important in scenarios where the classes are imbalanced, which is often the case in credit risk scorecards.

#### Data Split

In [22]:
# Split data into train and test 
X = df_prep_1.drop(target_column, axis = 1)
y = df_prep_1[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 42, stratify = y)

# Concatenate X_train with y_train to form df_train
df_train_1 = pd.concat([X_train, y_train], axis=1)

# Concatenate X_test with y_test to form df_test
df_test_1 = pd.concat([X_test, y_test], axis=1)

## Exploratory Data Analysis 

#### Validate Train Data 1

In [23]:
from validmind.tests.data_validation.TabularNumericalHistograms import TabularNumericalHistograms

vm_df = vm.init_dataset(dataset=df_train_1,
                                target_column=target_column)
test_context_train_1 = TestContext(dataset=vm_df)

metric = TabularNumericalHistograms(test_context_train_1)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-03 15:41:55,843 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-03 15:41:55,845 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [24]:
from validmind.tests.data_validation.HighCardinality import HighCardinality
metric = HighCardinality(test_context_train_1)
metric.run()
await metric.result.log()
metric.result.show()

In [25]:
from validmind.tests.data_validation.TabularCategoricalBarPlots import TabularCategoricalBarPlots
metric = TabularCategoricalBarPlots(test_context_train_1)
metric.run()
await metric.result.log()
metric.result.show()

#### Process Data 

In [26]:
# Select rows where purpose is 'debt_consolidation' or 'credit_card'
df_train_1 = df_train_1[df_train_1['purpose'].isin(['debt_consolidation', 'credit_card'])]
df_test_1 = df_test_1[df_test_1['purpose'].isin(['debt_consolidation', 'credit_card'])]

# Remove rows where grade is 'F' or 'G'
df_train_1 = df_train_1[~df_train_1['grade'].isin(['F', 'G'])]
df_test_1 = df_test_1[~df_test_1['grade'].isin(['F', 'G'])]

# Remove rows where sub_grade starts with 'F' or 'G'
df_train_1 = df_train_1[~df_train_1['sub_grade'].str.startswith(('F', 'G'))]
df_test_1 = df_test_1[~df_test_1['sub_grade'].str.startswith(('F', 'G'))]

# Remove rows where home_ownership is 'OTHER', 'NONE', or 'ANY'
df_train_1 = df_train_1[~df_train_1['home_ownership'].isin(['OTHER', 'NONE', 'ANY'])]
df_test_1 = df_test_1[~df_test_1['home_ownership'].isin(['OTHER', 'NONE', 'ANY'])]

# Update train and test
df_train_2 = df_train_1.copy()
df_test_2 = df_test_1.copy()

In [27]:
from validmind.tests.data_validation.TargetRateBarPlots import TargetRateBarPlots

vm_df = vm.init_dataset(dataset=df_train_2, target_column=target_column)
test_context_train_2 = TestContext(dataset=vm_df)

# Configure the metric
params = {
    "default_column": target_column,
    "columns": None
}

metric = TargetRateBarPlots(test_context_train_2, params=params)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-03 15:42:55,721 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-03 15:42:55,723 - INFO(validmind.vm_models.dataset): Inferring dataset types...


The column default is correct and contains only 1 and 0.


#### Process Data 

In [28]:
drop_categorical_features = ['addr_state']
drop_numerical_features = ['total_rec_int', 'loan_amnt',
                           'funded_amnt_inv', 'dti', 'revol_util', 'total_pymnt', 
                           'total_pymnt_inv', 'last_pymnt_amnt']

In [29]:
# Drop variables from next analysis
df_train_3 = df_train_2.drop(columns = drop_categorical_features + drop_numerical_features, axis=1)

# Update df_test 
df_test_3 = df_test_2.drop(columns = drop_categorical_features + drop_numerical_features, axis=1)

In [30]:
from validmind.tests.data_validation.ChiSquaredFeaturesTable import ChiSquaredFeaturesTable

vm_df = vm.init_dataset(dataset=df_train_3, target_column=target_column)
test_context_train_3 = TestContext(dataset=vm_df)

cat_features = get_categorical_columns(df_train_3)
params = {"cat_features": cat_features,
          "p_threshold": 0.05}

metric = ChiSquaredFeaturesTable(test_context_train_3, params)
metric.run()
await metric.result.log() 
metric.result.show()

2023-08-03 15:43:06,285 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-03 15:43:06,286 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [31]:
from validmind.tests.data_validation.ANOVAOneWayTable import ANOVAOneWayTable

num_features = get_numerical_columns(df_train_3)
params = {"num_features": num_features,
          "p_threshold": 0.05}

metric = ANOVAOneWayTable(test_context_train_3, params)
metric.run()
await metric.result.log()
metric.result.show()

In [32]:
from validmind.tests.data_validation.PearsonCorrelationMatrix import PearsonCorrelationMatrix

params = {"declutter": False,
          "features": None,
          "fontsize": 13}

metric = PearsonCorrelationMatrix(test_context_train_3, params)
metric.run()
await metric.result.log()
metric.result.show()

In [33]:
from validmind.tests.data_validation.FeatureTargetCorrelationPlot import FeatureTargetCorrelationPlot

params = {"features": None}

metric = FeatureTargetCorrelationPlot(test_context_train_3, params)
metric.run()
await metric.result.log()
metric.result.show()

## Feature Engineering

#### Process Train Data I: Class Binning

#### Validate Dataset

In [34]:
from validmind.tests.data_validation.WOEBinTable import WOEBinTable

# Update vm dataset and test context
vm_df = vm.init_dataset(dataset=df_train_3, 
                              target_column=target_column)
test_context_train_3 = TestContext(dataset=vm_df)

# Run test
metric = WOEBinTable(test_context_train_3)
metric.run()
await metric.result.log()
woe_iv_dic = metric.result.metric.value['woe_iv']
metric.result.show()

2023-08-03 15:43:10,385 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-03 15:43:10,385 - INFO(validmind.vm_models.dataset): Inferring dataset types...


Running with breaks_adj: None
Performing binning with breaks_adj: None
[INFO] creating woe binning ...


In [35]:
# Set test parameters
params = {
    "breaks_adj": {
        "int_rate": [5,10,15]}  
     }

# Run test
metric = WOEBinTable(test_context_train_3, params)
metric.run()
await metric.result.log()
woe_iv_dic = metric.result.metric.value['woe_iv']
metric.result.show()

Running with breaks_adj: {'int_rate': [5, 10, 15]}
Performing binning with breaks_adj: {'int_rate': [5, 10, 15]}
[INFO] creating woe binning ...


In [36]:
from validmind.tests.data_validation.WOEBinPlots import WOEBinPlots

# Set test parameters
params = {
    "breaks_adj": {"int_rate": [5,10,15]},
    "fig_height": 500,
}

# Run test
metric = WOEBinPlots(test_context_train_3, params=params)
metric.run()
await metric.result.log()
metric.result.show()

[INFO] creating woe binning ...


#### Process Train Data II: Class Coarsing

#### Validate Train Data 3

## Model Training

#### Process Train Data III: Convert Dataset into WoW Values

In [37]:
# Compute WoE 
params = {
    "breaks_adj": {
        "int_rate": [5,10,15]}  
     }

metric = WOEBinTable(test_context_train_3, params=params)
metric.run()
woe_dic = metric.result.metric.value['woe_iv']
woe_df = pd.DataFrame(woe_dic)

Running with breaks_adj: {'int_rate': [5, 10, 15]}
Performing binning with breaks_adj: {'int_rate': [5, 10, 15]}
[INFO] creating woe binning ...


In [38]:
import pandas as pd
import scorecardpy as sc

def transform_woe_df(woe_df):
    # Select and rename columns
    transformed_df = woe_df[['variable', 'bin', 'count', 'count_distr', 'good', 'bad', 'badprob', 'woe', 'bin_iv', 'total_iv']].copy()
    transformed_df.rename(columns={
        'bin_iv': 'total_iv'
    }, inplace=True)
    
    # Create 'is_special_values' column (assuming there are no special values)
    transformed_df['is_special_values'] = False
    
    # Transform 'bin' column into interval format and store it in 'breaks' column
    transformed_df['breaks'] = transformed_df['bin'].apply(lambda x: '[-inf, %s)' % x if isinstance(x, float) else '[%s, inf)' % x)
    
    # Group by 'variable' to create bins dictionary
    bins = {}
    for variable, group in transformed_df.groupby('variable'):
        bins[variable] = group
    
    return bins


def convert_to_woe(df, woe_df, target_col):
    # Create bins from woe_df
    bins = transform_woe_df(woe_df)
    
    # Make sure we don't transform the target column
    if target_col in bins:
        del bins[target_col]
    
    # Apply the WoE transformation
    dt_woe = sc.woebin_ply(df, bins=bins)
    
    return dt_woe


In [39]:
df_train_4 = convert_to_woe(df_train_3, woe_df, target_col=target_column) 

# Update df_test
df_test_4 = convert_to_woe(df_test_3, woe_df, target_col=target_column) 

[INFO] converting into woe values ...
[INFO] converting into woe values ...


In [40]:
df_train_4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105815 entries, 101002 to 462837
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   default                  105815 non-null  int64  
 1   emp_length_woe           105815 non-null  float64
 2   annual_inc_woe           105815 non-null  float64
 3   home_ownership_woe       105815 non-null  float64
 4   total_acc_woe            105815 non-null  float64
 5   term_woe                 105815 non-null  float64
 6   grade_woe                105815 non-null  float64
 7   inq_last_6mths_woe       105815 non-null  float64
 8   open_acc_woe             105815 non-null  float64
 9   sub_grade_woe            105815 non-null  float64
 10  int_rate_woe             105815 non-null  float64
 11  Unnamed: 0_woe           105815 non-null  float64
 12  installment_woe          105815 non-null  float64
 13  verification_status_woe  105815 non-null  float64
 14 

In [41]:
import statsmodels.api as sm

# Add constant to X_train for intercept term
y_train = df_train_4[target_column]
X_train = df_train_4.drop(target_column, axis=1)
X_train = sm.add_constant(X_train)
df_train_4 = pd.concat([X_train, y_train], axis=1)

# Update df_test
y_test = df_test_4[target_column]
X_test = df_test_4.drop(target_column, axis=1)
X_test = sm.add_constant(X_test)
df_test_4 = pd.concat([X_test, y_test], axis=1)

#### Fit Model 1

In [42]:
# Define the model
model = sm.GLM(y_train, X_train, family=sm.families.Binomial())

# Fit the model
model_fit_glm = model.fit()

# Print out the statistics
print(model_fit_glm.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:               105815
Model:                            GLM   Df Residuals:                   105800
Model Family:                Binomial   Df Model:                           14
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -45535.
Date:                Thu, 03 Aug 2023   Deviance:                       91071.
Time:                        15:43:58   Pearson chi2:                 1.06e+05
No. Iterations:                     5   Pseudo R-squ. (CS):            0.07964
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

#### Process Train Data 4

In [43]:
features_to_drop = []
df_train_5 = df_train_4.drop(columns = features_to_drop, axis=1)

# Update df_test 
df_test_5 = df_test_4.drop(columns = features_to_drop, axis=1)

#### Fit Model 2 

In [44]:
# Create X_train and y_train
y_train = df_train_5[target_column]
X_train = df_train_5.drop(target_column, axis=1)

# Update df_test
y_test = df_test_5[target_column]
X_test = df_test_5.drop(target_column, axis=1)

# Define the model
model = sm.GLM(y_train, X_train, family=sm.families.Binomial())

# Fit the model
model_fit_glm = model.fit()

# Save the model and train dataset for PD development 
save_data = False
if save_data:
    save_model_and_df(model_fit_glm, df=df_train_5, base_filename='model_fit_glm_scorecard')

# Print out the statistics
print(model_fit_glm.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:               105815
Model:                            GLM   Df Residuals:                   105800
Model Family:                Binomial   Df Model:                           14
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -45535.
Date:                Thu, 03 Aug 2023   Deviance:                       91071.
Time:                        15:43:58   Pearson chi2:                 1.06e+05
No. Iterations:                     5   Pseudo R-squ. (CS):            0.07964
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

## Model Evaluation

#### Validate Model Fit 2

In [45]:
# Create VM dataset
vm_ds_train_5 = vm.init_dataset(dataset=df_train_5,
                        target_column=target_column)
vm_ds_test_5 = vm.init_dataset(dataset=df_test_5,
                        target_column=target_column)

# Create VM model
vm_model_glm = vm.init_model(
    model = model_fit_glm, 
    train_ds=vm_ds_train_5, 
    test_ds=vm_ds_test_5)

2023-08-03 15:43:58,471 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-03 15:43:58,473 - INFO(validmind.vm_models.dataset): Inferring dataset types...
2023-08-03 15:43:59,829 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-03 15:43:59,829 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [46]:
from validmind.tests.model_validation.statsmodels.RegressionCoeffsPlot import RegressionCoeffsPlot

test_context_model = TestContext(models = [vm_model_glm])

metric = RegressionCoeffsPlot(test_context_model)
metric.run()
await metric.result.log()
metric.result.show()

In [47]:
from validmind.tests.model_validation.statsmodels.RegressionModelsCoeffs import RegressionModelsCoeffs

metric = RegressionModelsCoeffs(test_context_model)
metric.run()
await metric.result.log()
metric.result.show()

In [48]:
from validmind.tests.model_validation.statsmodels.LogRegressionConfusionMatrix import LogRegressionConfusionMatrix

test_context_model = TestContext(model= vm_model_glm)

# Configure test parameters
params = {
    "cut_off_threshold": 0.5,
}

metric = LogRegressionConfusionMatrix(test_context_model, params)
metric.run()
await metric.result.log()
metric.result.show()

In [49]:
from validmind.tests.model_validation.statsmodels.RegressionROCCurve import RegressionROCCurve

metric = RegressionROCCurve(test_context_model)
metric.run()
await metric.result.log()
metric.result.show()

In [50]:
from validmind.tests.model_validation.statsmodels.GINITable import GINITable

metric = GINITable(test_context_model)
metric.run()
await metric.result.log() 
metric.result.show()

In [51]:
from validmind.tests.model_validation.statsmodels.LogisticRegPredictionHistogram import LogisticRegPredictionHistogram

# Configure test parameters
params = {
    "title": "Histogram of Probability of Default",
}

metric = LogisticRegPredictionHistogram(test_context_model, params)
metric.run()
await metric.result.log()
metric.result.show()

In [52]:
from validmind.tests.model_validation.statsmodels.LogisticRegCumulativeProb import LogisticRegCumulativeProb

# Configure test parameters
params = {
    "title": "Cumulative Probability of Default",
}

metric = LogisticRegCumulativeProb(test_context_model, params)
metric.run()
await metric.result.log()
metric.result.show()

In [53]:
from validmind.tests.model_validation.statsmodels.ScorecardHistogram import ScorecardHistogram

# Configure test parameters
params = {
    "target_score": 600,
    "target_odds": 50,
    "pdo": 20,
    "title": "Histogram of Credit Scores",
}

metric = ScorecardHistogram(test_context_model, params)
metric.run()
await metric.result.log()
metric.result.show()